In [ ]:
import time

start_time = time.time()

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

width = 299
height = 299
channels = 3

import sys
import tarfile
from six.moves import urllib
import os

print("Point 1 elapsed time is {:.2f}s".format(time.time()-start_time))

TF_MODELS_URL = "http://download.tensorflow.org/models"
INCEPTION_V3_URL = TF_MODELS_URL + "/inception_v3_2016_08_28.tar.gz"
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")

def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()

def fetch_pretrained_inception_v3(url=INCEPTION_V3_URL, path=INCEPTION_PATH):
    if os.path.exists(INCEPTION_V3_CHECKPOINT_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "inception_v3.tgz")
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    inception_tgz = tarfile.open(tgz_path)
    inception_tgz.extractall(path=path)
    inception_tgz.close()
    os.remove(tgz_path)

fetch_pretrained_inception_v3()

print("Point 2 elapsed time is {:.2f}s".format(time.time()-start_time))

import re

CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M|re.U)

def load_class_names():
    with open(os.path.join("datasets", "inception", "imagenet_class_names.txt"),"rb") as f:
        countent = f.read().decode("utf-8")
        return CLASS_NAME_REGEX.findall(countent)

class_names = load_class_names()

class_names[:5]

print("Point 3 elapsed time is {:.2f}s".format(time.time()-start_time))

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

test_image = mpimg.imread(os.path.join("images", "cnn", "Alaskan_Malamute.jpg"))[:,:,:channels]
#plt.imshow(test_image)
#plt.axis("off")
#plt.show()
test_image = prepare_image(test_image)

X_test = test_image.reshape(-1, height, width, channels)

print("Point 4 elapsed time is {:.2f}s".format(time.time()-start_time))

import tensorflow as tf

print("Point 5 elapsed time is {:.2f}s".format(time.time()-start_time))

from tensorflow.contrib.slim.nets import inception
print("Point 6 elapsed time is {:.2f}s".format(time.time()-start_time))

import tensorflow.contrib.slim as slim
print("Point 7 elapsed time is {:.2f}s".format(time.time()-start_time))

import numpy as np


reset_graph()

X = tf.placeholder(tf.float32, shape=[None, 299, 299, 3], name="X")
with slim.arg_scope(inception.inception_v3_arg_scope()):
    print("Point 7.1 elapsed time is {:.2f}s".format(time.time()-start_time))    
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training = False)
print("Point 7.2 elapsed time is {:.2f}s".format(time.time()-start_time))
predictions = end_points["Predictions"]
print("Point 7.3 elapsed time is {:.2f}s".format(time.time()-start_time))

probabilities = tf.nn.softmax(logits)
saver = tf.train.Saver()

print("Point 8 elapsed time is {:.2f}s".format(time.time()-start_time))



In [ ]:

with tf.Session() as sess:
    saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    print("Point 9 elapsed time is {:.2f}s".format(time.time()-start_time))

    predictions_val = predictions.eval(feed_dict={X: X_test})
    print("Point 10 elapsed time is {:.2f}s".format(time.time()-start_time))
    
    bb_val = probabilities.eval (feed_dict={X: X_test})

most_likely_class_index = np.argmax(predictions_val[0])
print(most_likely_class_index)

print(class_names[most_likely_class_index])

top_5 = np.argpartition(predictions_val[0], -5)[-5:]
top_5 = top_5[np.argsort(predictions_val[0][top_5])]

print("Point 11 elapsed time is {:.2f}s".format(time.time()-start_time))
for i in top_5:
    print("{0}: {1:.2f}%".format(class_names[i], 100 * predictions_val[0][i]))

print("Point 12 elapsed time is {:.2f}s".format(time.time()-start_time))    




In [ ]:
sys.path.append(r"/Users/yiwenliu/Desktop/Projects/PML/HandsOn/models/slim/preprocessing")
import inception_preprocessing

In [ ]:
#from preprocessing import inception_preprocessing
session = tf.Session()

image_size = inception.inception_v3.default_image_size
def transform_img_fn(path_list):
    out = []
    for f in path_list:
        a = open(f, 'rb').read()
        image_raw = tf.image.decode_jpeg(open(f, 'rb').read(), channels=3)
        print("type is {}".format(image_raw.dtype))
#        image = inception_preprocessing.preprocess_image(image_raw, image_size, image_size, is_training=False)
        image = preprocess_for_eval(image_raw, image_size, image_size)
        out.append(image)
    return session.run([out])[0]



In [ ]:
X_test = transform_img_fn(['images/cnn/cat.jpg'])

In [ ]:
def preprocess_for_eval(image, height, width,
                        central_fraction=0.875, scope=None):
  """Prepare one image for evaluation.
  If height and width are specified it would output an image with that size by
  applying resize_bilinear.
  If central_fraction is specified it would crop the central fraction of the
  input image.
  Args:
    image: 3-D Tensor of image. If dtype is tf.float32 then the range should be
      [0, 1], otherwise it would converted to tf.float32 assuming that the range
      is [0, MAX], where MAX is largest positive representable number for
      int(8/16/32) data type (see `tf.image.convert_image_dtype` for details)
    height: integer
    width: integer
    central_fraction: Optional Float, fraction of the image to crop.
    scope: Optional scope for name_scope.
  Returns:
    3-D float Tensor of prepared image.
  """
  with tf.name_scope(scope, 'eval_image', [image, height, width]):
    if image.dtype != tf.float32:
      image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    # Crop the central region of the image with an area containing 87.5% of
    # the original image.
    if central_fraction:
      image = tf.image.central_crop(image, central_fraction=central_fraction)

    if height and width:
      # Resize the image to the specified height and width.
      image = tf.expand_dims(image, 0)
      image = tf.image.resize_bilinear(image, [height, width],
                                       align_corners=False)
      image = tf.squeeze(image, [0])
      image = tf.subtract(image, 0.5)
      image = tf.multiply(image, 2.0)
    return image

In [ ]:
a = np.squeeze(predictions_val)

In [ ]:
predictions_val[0][172:176]

In [ ]:
bb_val.shape

In [ ]:
bb_val[0][726]

In [ ]:
test_image = mpimg.imread(os.path.join("images", "cnn", "test_image.jpeg"), format="jpeg")[:,:,:channels]
#plt.imshow(test_image)
#plt.axis("off")
#plt.show()
X_test = test_image.reshape(-1, height, width, channels)

In [ ]:
X_test.dtype


In [ ]:
X_test.shape

In [ ]:
X_test = tf.image.convert_image_dtype(X_test, dtype=tf.float32)

In [ ]:
X_test.dtype
X_test.shape

In [ ]:
image_raw2 = tf.image.decode_jpeg(open('images/cnn/test_image.jpeg', 'rb').read(), channels=3)

In [ ]:
image_raw2.shape

In [ ]:
X_test2 = transform_img_fn(['images/cnn/test_image.jpeg'])

In [ ]:
X_test2[0].dtype

In [ ]:
X_test2[0].shape

In [ ]:
from scipy.misc import imresize
def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width/height
    target_image_ratio = target_width/target_height
    crop_vertically = image_ratio<target_image_ratio
    crop_width = width if crop_vertically else int(height*target_image_ratio)
    crop_height = int(width/target_image_ratio) if crop_vertically else height
  
    #做出一個window，大小在1／1.0~1.2
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    image = image[y0:y1, x0:x1]
    
    #翻一下
    if np.random.rand() < 0.5:
        image = np.fliplr(image)
        
    image = imresize(image, (target_width, target_height))
    return image.astype(np.float32)/255
#    return image

In [ ]:
image_data = tf.gfile.FastGFile('images/cnn/cat.jpg', 'rb').read()

In [ ]:
len(image_data)